# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
import statsmodels.api as sm
import numpy as np
from sklearn.tree import DecisionTreeRegressor


In [81]:
df = pd.read_csv('previsao_de_renda.csv')

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [83]:
# Remover colunas não relevantes
df = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

# Preenchimento de valores ausentes (ex: média do tempo_emprego)
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].mean())

# Variável resposta
y = df['renda']

# Selecionar as variáveis preditoras (excluindo a renda)
X = df.drop(columns=['renda'])

# Converter variáveis categóricas em dummy (drop_first evita multicolinearidade)
X = pd.get_dummies(X, drop_first=True)

# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Escalonar as variáveis
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [84]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

print("Ridge Regression:")
for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train_scaled, y_train)
    y_pred = ridge.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    ridge_results[a] = r2
    print(f"Alpha: {a:>5} | R²: {r2:.4f}")

# Identificar o melhor alpha
best_alpha_ridge = max(ridge_results, key=ridge_results.get)
print(f"\nMelhor modelo Ridge: alpha = {best_alpha_ridge} com R² = {ridge_results[best_alpha_ridge]:.4f}")

Ridge Regression:
Alpha:     0 | R²: 0.2691
Alpha: 0.001 | R²: 0.2691
Alpha: 0.005 | R²: 0.2691
Alpha:  0.01 | R²: 0.2691
Alpha:  0.05 | R²: 0.2691
Alpha:   0.1 | R²: 0.2691

Melhor modelo Ridge: alpha = 0 com R² = 0.2691


In [85]:
lasso_results = {}

print("\nLasso Regression:")
for a in alphas:
    # Aumenta o número de iterações para garantir a convergência
    lasso = Lasso(alpha=a, max_iter=10000)
    lasso.fit(X_train_scaled, y_train)
    y_pred = lasso.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    lasso_results[a] = r2
    print(f"Alpha: {a:>5} | R²: {r2:.4f}")

# Identificar o melhor alpha para Lasso
best_alpha_lasso = max(lasso_results, key=lasso_results.get)
print(f"\nMelhor modelo LASSO: alpha = {best_alpha_lasso} com R² = {lasso_results[best_alpha_lasso]:.4f}")


Lasso Regression:


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.858e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Alpha:     0 | R²: 0.2691
Alpha: 0.001 | R²: 0.2691
Alpha: 0.005 | R²: 0.2691
Alpha:  0.01 | R²: 0.2691
Alpha:  0.05 | R²: 0.2691
Alpha:   0.1 | R²: 0.2691

Melhor modelo LASSO: alpha = 0 com R² = 0.2691


In [87]:
y_train = y_train.reset_index(drop=True)
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    """Realiza seleção de variáveis por procedimento stepwise."""
    included = list(initial_list)
    while True:
        changed = False
        # Etapa de inclusão
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando: {best_feature:30} p-valor: {best_pval:.6f}')
        # Etapa de exclusão
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        # Exclui a constante
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f'Removendo: {worst_feature:30} p-valor: {worst_pval:.6f}')
        if not changed:
            break
    return included

# Convertendo os arrays escalonados para DataFrame com os nomes das colunas
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

print("\nStepwise Selection:")
selected_features = stepwise_selection(X_train_df, y_train, verbose=True)
print("Variáveis selecionadas:", selected_features)

# Ajustando o modelo com as variáveis selecionadas
X_train_step = X_train_df[selected_features]
X_test_df = pd.DataFrame(X_test_scaled, columns=X_train.columns)
X_test_step = X_test_df[selected_features]

model_step = sm.OLS(y_train, sm.add_constant(X_train_step)).fit()
y_pred_step = model_step.predict(sm.add_constant(X_test_step))
r2_step = r2_score(y_test, y_pred_step)
print(f"R² do modelo Stepwise: {r2_step:.4f}")


Stepwise Selection:
Adicionando: tempo_emprego                  p-valor: 0.000000
Adicionando: sexo_M                         p-valor: 0.000000
Adicionando: tipo_renda_Empresário          p-valor: 0.000000
Adicionando: educacao_Superior completo     p-valor: 0.000002
Adicionando: tipo_renda_Pensionista         p-valor: 0.000040
Adicionando: idade                          p-valor: 0.000003
Variáveis selecionadas: ['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo', 'tipo_renda_Pensionista', 'idade']
R² do modelo Stepwise: 0.2683


In [88]:
# Criação de um novo recurso: quadrado da idade
X_train_quad = pd.DataFrame(X_train, copy=True)
X_test_quad = pd.DataFrame(X_test, copy=True)

X_train_quad['idade2'] = X_train_quad['idade'] ** 2
X_test_quad['idade2'] = X_test_quad['idade'] ** 2

# Converter novamente as variáveis categóricas
X_train_quad = pd.get_dummies(X_train_quad, drop_first=True)
X_test_quad = pd.get_dummies(X_test_quad, drop_first=True)

# Garantir que as colunas sejam iguais nos dois conjuntos
X_train_quad, X_test_quad = X_train_quad.align(X_test_quad, join='inner', axis=1)

# Escalonar os dados
scaler2 = StandardScaler()
X_train_quad_scaled = scaler2.fit_transform(X_train_quad)
X_test_quad_scaled = scaler2.transform(X_test_quad)

# Ajustar, por exemplo, um modelo Ridge com a nova base
ridge_improved = Ridge(alpha=0.005)
ridge_improved.fit(X_train_quad_scaled, y_train)
y_pred_improved = ridge_improved.predict(X_test_quad_scaled)
r2_improved = r2_score(y_test, y_pred_improved)
print(f"\nRidge com termo quadrático: R² = {r2_improved:.4f}")


Ridge com termo quadrático: R² = 0.2698


In [91]:
# Usando os dados originais (sem escalonamento, pois árvores não exigem)
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)
print(f"\nÁrvore de Regressão: R² = {r2_tree:.4f}")


Árvore de Regressão: R² = 0.2940
